In [40]:
import importlib
import extraction_functions_praat
import functions
importlib.reload(extraction_functions_praat)
importlib.reload(functions)

<module 'functions' from '/home/dene/rp2/functions.py'>

In [ ]:
from pathlib import Path
import parselmouth
from parselmouth.praat import call
from extraction_functions_praat import PP_f0_mean_murton, PP_f0_sd_murton, PP_jitter, PP_f0_mean, PP_f0_sd
from functions import load_best_segment

In [ ]:
audio_dir = Path('audio_files_pre/')
features_dir = Path('extracted_features/')
segments_dir = Path('audio_files_segments/')

In [ ]:
f0_min = 60
f0_max = 300
jitter_type = 'all' ## choose from 'local' 'abs' 'rap' 'ppq5' 'ddp' 'all'

In [ ]:
AVAILABLE_FEATURES = [
    'PP_F0', 'PP_F0_SD', 
    'PP_F0_M', 'PP_F0_SD_M', 
    'PP_DUR', 
    'PP_JIT',
] ## add more

selected_features_dict = {
    'PP_F0': True, 'PP_F0_SD': True, 
    'PP_F0_M': True, 'PP_F0_SD_M': True, 
    'PP_DUR': False,
    'PP_JIT': True,
} ## enable the ones you want

In [ ]:
def extract_features_vowels(audio_file, selected_features):
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0'] = PP_f0_mean(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD' in selected_features:
        extracted_features['PP_F0_SD'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_M' in selected_features:
        extracted_features['PP_F0_M'] = PP_f0_mean_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD_M' in selected_features:
        extracted_features['PP_F0_SD_M'] = PP_f0_sd_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_JIT' in selected_features:
        jitter_values = PP_jitter(audio_file, f0_min=f0_min, f0_max=f0_max, type=jitter_type)
        if jitter_type == 'all':
            extracted_features['PP_JIT_local'] = jitter_values[0]
            extracted_features['PP_JIT_abs'] = jitter_values[1]
            extracted_features['PP_JIT_rap'] = jitter_values[2]
            extracted_features['PP_JIT_ppq5'] = jitter_values[3]
            extracted_features['PP_JIT_ddp'] = jitter_values[4]
        else:
            extracted_features[f'PP_JIT_{jitter_type}'] = jitter_values
            
    return extracted_features


def extract_features_mpt(audio_file, selected_features):
    sound = parselmouth.Sound(audio_file)
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0_SD'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    if 'PP_F0_SD_M' in selected_features:
        extracted_features['PP_F0_SD_M'] = PP_f0_sd_murton(audio_file, f0_min=f0_min, f0_max=f0_max)
    # if 'PP_DUR' in selected_features:
        # extracted_features['PP_DUR'] = PP_duration(audio_file)
    
    return extracted_features


def extract_features_sentences(audio_file, selected_features):
    sound = parselmouth.Sound(audio_file)
    extracted_features = {}
    
    if 'PP_F0' in selected_features:
        extracted_features['PP_F0'] = PP_f0_sd(audio_file, f0_min=f0_min, f0_max=f0_max)
    
    return extracted_features

In [ ]:
def process_audio_files(audio_files_dir, selected_features, analyze_best_segment=False):

    patient_df = {}

    for file in features_dir.iterdir():             
        parts = file.replace("pre.wav", "").split("_")
        if len(parts) < 4 or len(parts) > 4:
            print(f"Unexpected named audio file: {file}")
            continue

        patient_id, day, exercise, take_letter = parts

        file_path = audio_files_dir / file
        
        if exercise == 'VOW':
            if analyze_best_segment:
                if file[-1].isdigit():
                    continue
                if file[-1].isalpha():
                    best_segment_path = segments_dir / file
                    load_best_segment(audio_file=file_path, output_path=best_segment_path, f0_min=f0_min, f0_max=f0_max)
                    features = extract_features_vowels(best_segment_path, selected_features)
            else:
                features = extract_features_vowels(file_path, selected_features)
        elif exercise == 'MPT':
            features = extract_features_mpt(file_path, selected_features)
        elif exercise == 'SNT':
            features = extract_features_sentences(file_path, selected_features)
            
        df_entry = {'day': int(day), **features}
        
        if patient_id not in patient_df:
            patient_df[patient_id] = {}
            
        if exercise not in patient_df[patient_id]:
            patient_df[patient_id][exercise] = []
        
        patient_df[patient_id][exercise].append(df_entry)

    output_path = features_dir / patient_id
    
    # for patient_id, exercises in patient_df.items():
    #     output_dir = f"{features_dir}_{patient_id}/"
    #     os.makedirs(output_dir, exist_ok=True)

    #     for exercise, data in exercises.items():
    #         df = pd.DataFrame(data)
    #         df.to_csv(os.path.join(output_dir, f"{exercise}_{selected_features_abrv}.csv"), index=False)

In [ ]:
process_audio_files(audio_dir, selected_features=selected_features_dict, analyze_best_segment=True)

Processing 1234567_0_VOW_1.wav...
Processing 1234568_0_VOW_1.wav...
